In [ ]:
import pandas as pd
import torch
from torch import nn

from torch.utils import data
import torchvision
import numpy as np
import time

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/diplom/EDApiplineNNdata - EDApiplineNNdata.csv'
                  , sep = ',',
                 header = 0)
copy_df = df.copy()
mixed_df = copy_df.sample(frac = 1)
X_train, Y_train, X_test, Y_test = torch.tensor(mixed_df.iloc[:110, :18].to_numpy()), torch.tensor(mixed_df.iloc[:110, 18:].to_numpy()), torch.tensor(mixed_df.iloc[110:, :18].to_numpy()), torch.tensor(mixed_df.iloc[110:, 18:].to_numpy())

In [ ]:
net = nn.Sequential(
    nn.Linear(18, 800),
    nn.ReLU(),
    nn.Linear(800, 400),
    nn.ReLU(),
    nn.Linear(400, 160),
    nn.Sigmoid(),
    nn.Linear(160, 16)
)

# функция ошибки
error = nn.CrossEntropyLoss()

# алгоритм оптимизации
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)



In [ ]:
# количество эпох
epoch_num = 200

In [ ]:
f1test_array = []
f1train_array = []
mean_time_lr = 0
mean_time_eval = 0
for epoch_idx in range(epoch_num):
    print('Epoch #{}'.format(epoch_idx))

    # переводим НС в состояние обучения
    net.train()
    
    train_correct = 0
    true_positive = 0
    false_negative = 0
    false_positive = 0
    # итерация по обучающим пакетам
    for data, target in zip(X_train, Y_train):
        t1 = time.perf_counter_ns()
        data = data.float()
        target = target.float()

        # обнуление матриц градиентов (коррекций) для всех слоев нейросети
        optimizer.zero_grad()
        # прямое распространение
        pred = net(data)
        # вычисление ошибки
        loss = error(pred, target)
        # обратное распространение
        loss.backward()
        # обновление параметров
        optimizer.step()

        t2 = time.perf_counter_ns()
        mean_time_lr += t2 - t1
        # вычисление и запись количества корректных распознаваний
        pred_labels = torch.tensor(list(map(lambda label : 1.0 if label > 3 else 0.0, pred))).float()
        train_correct += (pred_labels==target).sum().item()
        true_positive += ((pred_labels==target)&(pred_labels==1)).sum().item()
        false_positive += ((pred_labels!=target)&(pred_labels==1)).sum().item()
        false_negative += ((pred_labels!=target)&(pred_labels==0)).sum().item()
        # запись значения ошибки
    if true_positive > 0:
      presision = true_positive/(true_positive + false_positive)
      recall = true_positive/(true_positive + false_negative)
      f1train = 2*presision*recall/(presision + recall)
      print('f1 = {}'.format(f1train))
      f1train_array.append(f1train)
    else: 
      print("f1 = 0")
      f1train_array.append(0)
    print('Train acccuracy = {}'.format(train_correct/(X_train.size(0)*X_train.size(1))))

    # test step
    # переводим НС в состояние тестирования
    net.eval()

    # задаем переменные, куда будет записано количество верных распознаваний
    # и значение ошибки на эпохе
    test_correct = 0
    test_true_positive = 0
    test_false_negative = 0
    test_false_positive = 0
    # контекст torch.no_grad() запрещает вычисление градиентов, что ускоряет процедуру тестирования
    with torch.no_grad():
        # итерация по тестовым пакетам (batch iteration)
        for data, target in zip(X_test, Y_test):
            t1 = time.perf_counter_ns()
            data = data.float()
            target = target.float()
            # вычисление ошибки
            pred = net(data)
            
            t2 = time.perf_counter_ns()
        
            mean_time_eval += t2 - t1


            # вычисление и запись количества корректных распознаваний
            pred_labels = torch.tensor(list(map(lambda label : 1.0 if label > 3 else 0.0, pred))).float()
            test_true_positive += ((pred_labels==target)&(pred_labels==1)).sum().item()
            test_false_positive += ((pred_labels!=target)&(pred_labels==1)).sum().item()
            test_false_negative += ((pred_labels!=target)&(pred_labels==0)).sum().item()
            
            test_correct += (pred_labels==target).sum().item()
        if test_true_positive > 0:
          test_presision = test_true_positive/(test_true_positive + test_false_positive)
          test_recall = test_true_positive/(test_true_positive + test_false_negative)
          f1_test = 2*test_presision*test_recall/(test_presision + test_recall)
        else: f1_test = 0
        print('f1 score test = {}'.format(f1_test))
        print('Test acccuracy = {}'.format(test_correct/(X_test.size(0)*X_test.size(1))))
        print('---------------------')
        f1test_array.append(f1_test)
        if f1_test > 1:
          break
print("Mean time train: ", mean_time_lr/((epoch_idx + 1)*X_train.size(0)))
print("Mean time eval: ", mean_time_eval/((epoch_idx + 1)*X_test.size(0)))

Epoch #0
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #1
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #2
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #3
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #4
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #5
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #6
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666666666666
---------------------
Epoch #7
f1 = 0
Train acccuracy = 0.6666666666666666
f1 score test = 0
Test acccuracy = 0.6666666

In [ ]:
np.savetxt(fname="f1_train.csv", delimiter=",", X=f1train_array, fmt='%10.5f')
np.savetxt(fname="f1_test.csv", delimiter=",", X=f1test_array, fmt='%10.5f')